# Lightweight Fine-Tuning Project

In this cell, describe your choices for each of the following

* PEFT technique: 
LoRA (Low-Rank Adaptation) technique was used

* Model: 
GPT2 Model was used as a base model

* Evaluation approach: 
evaluate method of Trainer class from Huggging Face was chosen
More Specifically, compute_metrics method was overloaded which takes the mean of predicted value vs actual value

* Fine-tuning dataset: duxprajapati/symptom-disease-dataset was chosen
This is a data set which comprises symptoms in the "text" column and the actual disease in the "label" column; The objective is to predict a disease based on the symptoms

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [2]:
!pip install -U datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 1.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 1.4 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
from datasets import load_dataset


# Load the train and test splits of the duxprajapati/symptom-disease-dataset
splits = ["train", "test"]
dataset = {split: ds for split, ds in zip(splits, load_dataset("duxprajapati/symptom-disease-dataset", split=splits))}

# Show the dataset
dataset



Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 5634
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 1409
 })}

In [3]:
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification, AutoModelForSequenceClassification
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification, Trainer, TrainingArguments

# Load the train and test splits of the duxprajapati/symptom-disease-dataset
splits = ["train", "test"]
dataset = {split: ds for split, ds in zip(splits, load_dataset("duxprajapati/symptom-disease-dataset", split=splits))}

unique_labels = set(dataset["train"]["label"]).union(set(dataset["test"]["label"]))
num_labels = len(unique_labels)
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {v: k for k, v in label_to_id.items()}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Define a custom dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = [label_to_id[label] for label in labels]  # Encode labels
        self.tokenizer = tokenizer
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# Load the pre-trained tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels = num_labels)



# Set the padding token and padding side
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id
model.id_to_label = id_to_label
model.label_to_id = label_to_id

# Set the model to evaluation mode
#model.eval()

# Initialize the Trainer
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,

)

/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import pandas as pd
import numpy as np


# Example text and labels
texts = dataset["test"]["text"][:10]
labels = dataset["test"]["label"][:10]


# Tokenize and encode the text
test_dataset = TextClassificationDataset(texts, labels, tokenizer)

# Inference
results = trainer.predict(test_dataset)


df = pd.DataFrame(
    {
        "text": [item for item in texts],
        "predictions": results.predictions.argmax(axis=1),
        "labels": results.label_ids,
    }
)
# Show all the cell
pd.set_option("display.max_colwidth", None)
df

,text,predictions,labels
0,"itching,vomiting,fatigue,weight_loss,high_fever,yellowish_skin,dark_urine,abdominal_pain",1032,541
1,"fatigue,weight_loss,restlessness,sweating,diarrhoea,fast_heart_rate,excessive_hunger,muscle_weakness,irritability,abnormal_menstruation",283,502
2,"constipation,pain_during_bowel_movements,pain_in_anal_region,bloody_stool,irritation_in_anus",1032,297
3,"chills,vomiting,fatigue,weight_loss,cough,high_fever,sweating,loss_of_appetite,mild_fever,yellowing_of_eyes,swelled_lymph_nodes,malaise,phlegm,chest_pain,blood_in_sputum",184,1019
4,"vomiting,yellowish_skin,abdominal_pain,swelling_of_stomach,distention_of_abdomen,history_of_alcohol_consumption,fluid_overload",1032,33
5,"vomiting,breathlessness,sweating,chest_pain",1032,447
6,"stomach_pain,acidity,ulcers_on_tongue,vomiting,cough,chest_pain",1032,412
7,"skin_rash,chills,joint_pain,vomiting,fatigue,high_fever,headache,nausea,loss_of_appetite,pain_behind_the_eyes,back_pain,malaise,muscle_pain,red_spots_over_body",986,275
8,"vomiting,headache,altered_sensorium",1032,718
9,"burning_micturition,bladder_discomfort,continuous_feel_of_urine",345,1035


In [12]:

def compute_metrics(predictions, actuals):
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    return {"accuracy": (predictions == actuals).mean()}


compute_metrics(results.predictions.argmax(axis=1), results.label_ids)

{'accuracy': 0.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [8]:
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoModelForCausalLM
# Load the dataset

# Load the train and test splits of the duxprajapati/symptom-disease-dataset
splits = ["train", "test"]
dataset = {split: ds for split, ds in zip(splits, load_dataset("duxprajapati/symptom-disease-dataset", split=splits))}


dataset

unique_labels = set(dataset["train"]["label"]).union(set(dataset["test"]["label"]))
num_labels = len(unique_labels)
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {v: k for k, v in label_to_id.items()}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Define a custom dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = [label_to_id[label] for label in labels]  # Encode labels
        self.tokenizer = tokenizer
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Load the tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=num_labels)

# Set the padding token and padding side
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Fine Tune the model
config = LoraConfig(
task_type=TaskType.SEQ_CLS)
lora_model = get_peft_model(model, config)
lora_model.print_trainable_parameters()
lora_model.config.pad_token_id = tokenizer.eos_token_id
lora_model.id_to_label = id_to_label
lora_model.label_to_id = label_to_id

# Create datasets
train_dataset = TextClassificationDataset(dataset["train"]["text"], dataset["train"]["label"], tokenizer)
eval_dataset = TextClassificationDataset(dataset["test"]["text"], dataset["test"]["label"], tokenizer)


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
)

# Initialize the Trainer
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print("Evaluation results: {eval_results}")

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,956,864 || all params: 126,396,672 || trainable%: 1.5481926612751322


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,4.750000,4.587873,0.201561
2,4.056400,4.159091,0.386089
3,4.053000,3.944731,0.414478


Evaluation results: {'eval_loss': 3.9447312355041504, 'eval_accuracy': 0.41447835344215755, 'eval_runtime': 25.7866, 'eval_samples_per_second': 54.641, 'eval_steps_per_second': 6.864, 'epoch': 3.0}


In [10]:
lora_model.save_pretrained("gpt-lora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [7]:
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification, AutoModelForSequenceClassification
import torch
from torch.utils.data import Dataset
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, TaskType, PeftModel
from transformers import GPT2TokenizerFast, GPT2ForSequenceClassification, Trainer, TrainingArguments

# Load the train and test splits of the duxprajapati/symptom-disease-dataset
splits = ["train", "test"]
dataset = {split: ds for split, ds in zip(splits, load_dataset("duxprajapati/symptom-disease-dataset", split=splits))}


unique_labels = set(dataset["train"]["label"]).union(set(dataset["test"]["label"]))
num_labels = len(unique_labels)
label_to_id = {label: i for i, label in enumerate(unique_labels)}
id_to_label = {v: k for k, v in label_to_id.items()}

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Define a custom dataset class
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.texts = texts
        self.labels = [label_to_id[label] for label in labels]  # Encode labels
        self.tokenizer = tokenizer
        self.encodings = tokenizer(texts, truncation=True, padding='max_length', max_length=max_length)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


# Load the pre-trained tokenizer and saved model
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = AutoModelForSequenceClassification.from_pretrained('gpt2', num_labels = num_labels)

# Set the model to evaluation mode
#model.eval()

# Set the padding token and padding side
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

# Fine Tune the model
config = LoraConfig.from_pretrained("gpt-lora")
#lora_model = get_peft_model(model, config)
lora_model = PeftModel.from_pretrained(model, "gpt-lora", peft_config=config)
lora_model.print_trainable_parameters()
lora_model.config.pad_token_id = tokenizer.eos_token_id
lora_model.id_to_label = id_to_label
lora_model.label_to_id = label_to_id

lora_model.eval()

# Set the padding token and padding side
tokenizer.padding_side = "left"  # Pad on the left for GPT2
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id


# Initialize the Trainer
trainer = Trainer(
    model=lora_model,
    tokenizer=tokenizer,

)



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 1,661,952 || all params: 126,396,672 || trainable%: 1.314870062401643


In [8]:
import pandas as pd
import numpy as np


# Example text and labels
texts = dataset["test"]["text"][:100]
labels = dataset["test"]["label"][:100]


# Tokenize and encode the text
test_dataset = TextClassificationDataset(texts, labels, tokenizer)

# Inference
results = trainer.predict(test_dataset)


def compute_metrics(predictions, actuals):
    predictions = np.array(predictions)
    actuals = np.array(actuals)
    return {"accuracy": (predictions == actuals).mean()}


compute_metrics(results.predictions.argmax(axis=1), results.label_ids)


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'accuracy': 0.36}